In [1]:
# Import libraties and tools
import os
import sys
import numpy as np
import datetime
import glob
import skimage.morphology
import matplotlib.pyplot as plt

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5 as as5, acweRestoreScale as ars
from DatasetTools import DataManagmentTools as dmt
from SDO_tools import read_fits

In [ ]:
# Key Varibles

# Datset
dataFolder  = '/home/jgra/Coronal Holes/DailyCadenceData/'
traceFolder = os.path.join(ROOT_DIR, 'DatasetTools/DailyCadenceDownloadLists/')

# SaveFolder - Update to refect location where data will be saved 
saveFolder = '/mnt/coronal_holes/Paper 2/Code 02 Observations'
saveFolder = saveFolder + '/FinalPipeline/DailyCadence/Standard/Uni/'

daterange = np.array([datetime.datetime(2024,5,1,0,0,0),
                      datetime.datetime(2024,6,1,0,0,0)])

strell = 40

In [ ]:
# List of data

CRsaveFolders = sorted(glob.glob(saveFolder + '*'))

In [ ]:
# Cycle throught Dataset

for i in range(len(CRsaveFolders)):
    
    CRsaveFolder = CRsaveFolders[i]
    
    # Get CR
    CR = os.path.basename(CRsaveFolder)
    
    # Get List of images
    files = sorted(glob.glob(CRsaveFolder + '/*/*.npz'))
    
    # Continue only if in date range
    for file in files:
        
        # Corrisponding Files
        folder  = CR + '/' + file.split('/')[-2] + '/'
        euvFile = glob.glob(dataFolder + folder + 'aia.*.193.*.fits')[0]
        hmiFile = glob.glob(dataFolder + folder + 'hmi.*.fits')[0]
        
        # Continue if in date range
        obDate =  dmt.timeFromFilenameAIA(euvFile)
        if obDate >= daterange[0] and obDate <= daterange[1]:
            
            # Open Segmentation
            Hseg,AH,SEG = as5.openSeg(file)
            SEG = ars.upscale(SEG,AH)
            
            # Open EUV Data
            Ieuv,IeuvDsp,Heuv = read_fits.openAIA(euvFile)
            
            # Open HMI Data
            Ihmi,Hhmi = read_fits.openHMI(hmiFile,Ieuv,Heuv)
            IhmiDsp = np.clip(Ihmi,-100,100)
            
            # Find and Label CH Regions
            ACWE_clusters = SEG * 1
            ACWE_clusters[np.where(ACWE_clusters>0)] = 1
            ACWE_clusters = skimage.morphology.dilation(ACWE_clusters.astype(bool),
                                                        np.ones([strell,strell]))
            ACWE_clusters = skimage.measure.label(ACWE_clusters,connectivity=2)
            
            # Plot Cases
            plt.figure(figsize=[5,2],dpi=300)
            plt.rcParams.update({'font.size': 6})
            title = Heuv['T_REC'] + ', ' + str(np.max(ACWE_clusters)) +\
                    ' Region(s)\nExposure Time: ' + str(Heuv['EXPTIME']) + ' s'
            plt.suptitle(title)
            plt.subplot(1,3,1)
            plt.imshow(np.flip(IeuvDsp,axis=0),cmap='gray')
            plt.axis(False)
            plt.title('EUV Observation')
            plt.subplot(1,3,2)
            plt.imshow(np.flip(SEG,axis=0),interpolation='None')
            plt.axis(False)
            plt.title('Segmentation')
            plt.subplot(1,3,3)
            plt.imshow(np.flip(IhmiDsp,axis=0),cmap='gray')
            plt.axis(False)
            plt.title('HMI Observation')
            plt.show()